# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

#### Details Student 1: 
name: Bar Elimelech  
ID: 313586869  
email: elimelechmail@gmail.com  

#### Details Student 2: 
name: Nitzan Kohan  
ID: 316517390  
email: nitzank1995@gmail.com  

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [2]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [3]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [4]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [5]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [6]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [7]:
# Hebrew tokenizer import:

# unmark if you want to use:
# import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [8]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [9]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [10]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

### Clean dataset

In [11]:
def remove_non_heb(text):    
    hebrew_pattern = re.compile(r'[^\u05D0-\u05EA\s]+')
    return hebrew_pattern.sub('', text)

def clean_dataset(df):
    clean_df = df.copy()
    clean_df.dropna(inplace=True)
    clean_df.drop_duplicates(["story"], inplace=True)
    clean_df['story'] = clean_df['story'].apply(lambda x: remove_non_heb(x))
    
    return clean_df

In [12]:
clean_df = clean_dataset(df_train)

In [13]:
clean_df.head(15)

,story,gender
0,כשחבר הזמין אותי לחול לא באמת חשבתי שזה יקרה פ...,m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,כשהייתי ילד מטוסים היה הדבר שהכי ריתק אותי בתו...,m
4,הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכים...,f
5,לפני כ חודשים טסתי לרומא למשך שבוע טסתי במטוס ...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב הדירה הי...,f
8,עד לפני כמה חודשים אני הייתי בטוח שכל חיי אהיה...,m
9,בן זוגי חגג יום הולדת ורציתי להפתיע אותו ולסגו...,f


In [14]:
X = clean_df['story']
y = clean_df['gender']

### Create vercorizers

In [15]:
count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(X)

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(X)

### Split train test for both vectorizer matrix

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    count_matrix, 
    y, 
    test_size=0.2, 
    random_state=42
)

In [17]:
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(
    tfidf_matrix, 
    y, 
    test_size=0.2, 
    random_state=42
)

### Define classifier params

In [18]:
dt_param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

knn_param_grid = {
    "n_neighbors": [3, 5, 7, 9, 11],
    "weights": ["uniform", "distance"],
    "p": [1, 2],
}

svc_param_grid = {
    'C': [0.1, 1, 10]
}

lr_param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2']
}

### Init classifiers

In [19]:
dt_classifier = DecisionTreeClassifier()
knn_classifier = KNeighborsClassifier()
svc_classifier = LinearSVC()
lr_classifier = linear_model.LogisticRegression()

### Grid search

In [20]:
dt_grid_search = GridSearchCV(estimator=dt_classifier, param_grid=dt_param_grid, scoring=metrics.make_scorer(f1_score, average='macro'), cv=10, n_jobs=-1)
knn_grid_search = GridSearchCV(estimator=knn_classifier, param_grid=knn_param_grid, scoring=metrics.make_scorer(f1_score, average='macro'), cv=10, n_jobs=-1)
svc_grid_search = GridSearchCV(estimator=svc_classifier, param_grid=svc_param_grid, scoring=metrics.make_scorer(f1_score, average='macro'), cv=10, n_jobs=-1)
lr_grid_search = GridSearchCV(estimator=lr_classifier, param_grid=lr_param_grid, scoring=metrics.make_scorer(f1_score, average='macro'), cv=10, n_jobs=-1)

### Fit - counterVectorizer

In [21]:
counter_dt_res = dt_grid_search.fit(X_train, y_train)
counter_knn_res = knn_grid_search.fit(X_train, y_train)
counter_svc_res = svc_grid_search.fit(X_train, y_train)
counter_lr_res = lr_grid_search.fit(X_train, y_train)

### Fit - TfidfVectorizer

In [22]:
tfidf_dt_res = dt_grid_search.fit(X_train_tfidf, y_train_tfidf)
tfidf_knn_res = knn_grid_search.fit(X_train_tfidf, y_train_tfidf)
tfidf_svc_res = svc_grid_search.fit(X_train_tfidf, y_train_tfidf)
tfidf_lr_res = lr_grid_search.fit(X_train_tfidf, y_train_tfidf)

### Compare results

In [23]:
tfidf_grid_list = [tfidf_dt_res, tfidf_knn_res, tfidf_svc_res, tfidf_lr_res]
counter_grid_list = [counter_dt_res, counter_knn_res, counter_svc_res, counter_lr_res]

print("TfidfVectorizer RESULTS:")
for grid in tfidf_grid_list:
    print(f"\nESTIMATOR: {grid.estimator}")
    print(f"- BEST PARAMS: {grid.best_params_}")
    print(f"- BEST SCORE: {grid.best_score_}")
    print(f"- PREDICTION SCORE: {f1_score(y_test_tfidf, grid.predict(X_test_tfidf), average='macro')}")
    
print("\ncounterVectorizer RESULTS:")
for grid in counter_grid_list:
    print(f"\nESTIMATOR: {grid.estimator}")
    print(f"- BEST PARAMS: {grid.best_params_}")
    print(f"- BEST SCORE: {grid.best_score_}")
    print(f"- PREDICTION SCORE: {f1_score(y_test, grid.predict(X_test), average='macro')}")

TfidfVectorizer RESULTS:

ESTIMATOR: DecisionTreeClassifier()
- BEST PARAMS: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10}
- BEST SCORE: 0.645825646981171
- PREDICTION SCORE: 0.5837187789084182

ESTIMATOR: KNeighborsClassifier()
- BEST PARAMS: {'n_neighbors': 5, 'p': 2, 'weights': 'distance'}
- BEST SCORE: 0.5638382306869005
- PREDICTION SCORE: 0.6340471672539983

ESTIMATOR: LinearSVC()
- BEST PARAMS: {'C': 10}
- BEST SCORE: 0.5388530077965117
- PREDICTION SCORE: 0.6803977272727273

ESTIMATOR: LogisticRegression()
- BEST PARAMS: {'C': 10, 'penalty': 'l2'}
- BEST SCORE: 0.4510013322758331
- PREDICTION SCORE: 0.4752798507462686

counterVectorizer RESULTS:

ESTIMATOR: DecisionTreeClassifier()
- BEST PARAMS: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10}
- BEST SCORE: 0.645825646981171
- PREDICTION SCORE: 0.4676068335620652

ESTIMATOR: KNeighborsClassifier()
- BEST PARAMS: {'n_neighbors': 5, 'p': 2, 'weights':

### Apply best result classifier on test data

In [24]:
df_test.head()

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...
3,3,"רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני..."
4,4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...


In [25]:
# Iinit LinearSVC with best param - {C: 10}
best_svc_classifier = LinearSVC(C=10)
best_svc_classifier.fit(X_train, y_train)

# Remove non hebrew characters from df_test stories  
df_test['story'] = df_test['story'].apply(lambda x: remove_non_heb(x))
new_count_matrix = count_vectorizer.transform(df_test['story'])
best_y_pred = best_svc_classifier.predict(new_count_matrix)

LinearSVC(C=10)

In [26]:
df_predicted = pd.DataFrame({'test_example_id': df_test['test_example_id'], 'predicted_category':best_y_pred})

### Gender distribution - train VS predict

In [27]:
df_train.gender.value_counts()
df_predicted.predicted_category.value_counts()

m    575
f    178
Name: gender, dtype: int64

m    263
f     60
Name: predicted_category, dtype: int64

In [28]:
display(df_predicted.head(5), df_predicted.tail(5))

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,m


,test_example_id,predicted_category
318,318,m
319,319,m
320,320,m
321,321,m
322,322,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [29]:
df_predicted.to_csv('classification_results.csv',index=False)

/Users/bar-study/opt/anaconda3/envs/pyenv3.8/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/bar-study/opt/anaconda3/envs/pyenv3.8/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/bar-study/opt/anaconda3/envs/pyenv3.8/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/bar-study/opt/anaconda3/envs/pyenv3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHE

/Users/bar-study/opt/anaconda3/envs/pyenv3.8/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/bar-study/opt/anaconda3/envs/pyenv3.8/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/bar-study/opt/anaconda3/envs/pyenv3.8/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/bar-study/opt/anaconda3/envs/pyenv3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHE

/Users/bar-study/opt/anaconda3/envs/pyenv3.8/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/bar-study/opt/anaconda3/envs/pyenv3.8/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/bar-study/opt/anaconda3/envs/pyenv3.8/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/bar-study/opt/anaconda3/envs/pyenv3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHE